In [79]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random
import boto3
from datetime import datetime
import re

In [80]:
session = boto3.Session(profile_name='default')
s3 = session.client('s3')
bucket = '2018102232'

#### Crawling Coupang

In [104]:
baseUrl = "https://www.coupang.com/np/categories/176530?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page={page}&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176430&rating=0"
ids = []
products = []
new_prices = []
old_prices = []
images = []
href = []

page = 1
while page < 10 :
    url = baseUrl.format(page=page)
    print("page :%d" %page)
    try : 
        res = requests.get(url, timeout=10)
    except requests.exceptions.Timeout as e :
        continue
    time.sleep(random.uniform(2,4))
    soup = BeautifulSoup(res.text, 'html.parser')
    product_list = soup.find_all('li', class_='baby-product renew-badge')
    
    ### 각 요소에 대해 아이디, 상품 이름, 용량, 개수 및 썸네일 이미지 파싱
    for product in product_list :
        link= product.find('a', class_='baby-product-link')
        ids.append(int(link['data-item-id']))
        href.append('https://www.coupang.com'+link['href'])
        images.append('https:'+link.find('img')['src'])
        products.append(product.find('div', class_='name').text.strip())
        if product.find('del', class_='base-price'):
            origin_price = re.sub(',','',product.find('del', class_='base-price').text.strip())
        else :
            origin_price = None
        dis_price = re.sub(',','',product.find('strong', class_='price-value').text.strip())
        old_prices.append(origin_price)
        new_prices.append(dis_price)
        

        
    time.sleep(random.uniform(2,4))
    print(url)
    page += 1

page :1
page :1
page :1
https://www.coupang.com/np/categories/176530?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176430&rating=0
page :2
page :2
page :2
page :2
page :2
https://www.coupang.com/np/categories/176530?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176430&rating=0
page :3
page :3
page :3
page :3
https://www.coupang.com/np/categories/176530?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176430&rating=0
page :4
https://www.coupang.com/np/categories/176530?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=4&channel=user&fromComponent=N&selected

In [105]:
name = []
volume = []
quantity = []

for product in products : 
    splited = product.split(',')
    if len(splited) == 1:
        name.append(product)
        volume.append(None)
        quantity.append(None)
    elif len(splited) == 2: 
        name.append(splited[0])
        volume.append(None)
        quantity.append(splited[1])
    else :
        cur_volume = re.sub(r'[^0-9]', '',splited[1])
        cur_quantity = re.sub(r'[^0-9]','',splited[2])
        if cur_volume and cur_quantity: 
            name.append(splited[0])
            volume.append(int(cur_volume)*int(cur_quantity))
            quantity.append(int(cur_quantity))
        elif cur_quantity : 
            name.append(splited[0])
            volume.append(None)
            quantity.append(int(cur_quantity))
        elif cur_volume :
            name.append(splited[0])
            volume.append(int(cur_volume))
            quantity.append(None)
        else :
            name.append(product)
            volume.append(None)
            quantity.append(None)

In [106]:
df = pd.DataFrame(list(zip(ids, name, volume, quantity,old_prices, new_prices, images, href)), index=range(len(products)), columns=['id', 'name','volume','quantity','old_price','new_price', 'img_src', 'href'])

In [107]:
df.tail(5)

,id,name,volume,quantity,old_price,new_price,img_src,href
1075,6091120175,닥터 반코르 세라마이드 크림,60.0,1,None,18750,https://thumbnail7.coupangcdn.com/thumbnails/r...,https://www.coupang.com/vp/products/5825089072...
1076,7344184612,아이오페 스템 3 에멀젼,145.0,1,44000,21860,https://thumbnail9.coupangcdn.com/thumbnails/r...,https://www.coupang.com/vp/products/5225684017...
1077,14008542014,엘로엘 선쿠션 팡팡 빅선쿠션 S6 본품+리필,25.0,1,None,39900,https://thumbnail6.coupangcdn.com/thumbnails/r...,https://www.coupang.com/vp/products/6451628382...
1078,14124234763,XINDIBO ZOOSON 반점 제거 연고 주근깨 반점 멜라닌 흑반 황갈반 등 모두...,NaN,None,None,34000,https://thumbnail8.coupangcdn.com/thumbnails/r...,https://www.coupang.com/vp/products/6471033697...
1079,2539722274,오휘 미라클 모이스처 에멀전,140.0,1,None,20360,https://thumbnail6.coupangcdn.com/thumbnails/r...,https://www.coupang.com/vp/products/1478286381...


In [108]:
today = datetime.today().strftime("%Y%m%d")
df.to_csv("coupang{today}.csv".format(today=today), index=False)
s3.upload_file('coupang{today}.csv'.format(today=today), '2018102232','coupang/{today}.csv'.format(today=today))

#### Crawling OliveYoung

In [109]:
olive_url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx={index}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010008_Small'

In [110]:
ids = []
name = []
volumes = []
old_prices = []
new_prices = []
images = []
href = []

for ind in range(1, 16) :
    url = olive_url.format(index=ind)
    res = requests.get(url)

    soup = BeautifulSoup(res.text, 'html.parser')

    products = soup.find_all("div", class_='prd_info')

    for product in products :
        url = product.find("a")
        product_url = url['href']
        p_id = url['data-ref-goodsno']
        img = url.find('img')['src']
        brand = product.find("span", class_='tx_brand').text
        p_name = product.find("p", class_="tx_name").text.strip()
        
        if product.find('span',class_='tx_org'):
            before_discount = int(re.sub(',','',product.find('span', class_='tx_org').find('span').text))
        else :
            before_discount = None
        discounted = int(re.sub(',','',product.find('span', class_='tx_cur').find('span').text))

        product_name = re.sub(r'\[[^)]*\]'," ",p_name)
        product_name = re.sub(r'\([^)]*\)',' ',product_name).strip()
        splited_name = re.sub(r'[()]',' ',re.sub(r'[+]',' ',p_name)).split()
        volume = 0
        for split in splited_name :
            if split[-2:] == 'ml' :
                volume += int(re.sub('[^0-9]','',split))

        ids.append(p_id)
        name.append(product_name)
        volumes.append(volume)
        old_prices.append(before_discount)
        new_prices.append(discounted)
        images.append(img)
        href.append(product_url)


In [111]:
olive_df = pd.DataFrame(list(zip(ids, name, volumes,old_prices, new_prices, images, href)), index=range(len(ids)), columns=['id', 'name','volume','old_price','new_price', 'img_src', 'href'])

In [112]:
olive_df.head(5)

,id,name,volume,old_price,new_price,img_src,href
0,A000000162254,아누아 어성초 77 수딩 토너 350ml 기획,400,30500.0,22800,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000155253,아비브 어성초 카밍 토너 스킨부스터 더블 기획,400,39000.0,27300,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000148123,아이디얼 포맨 시카 올인원 기획,200,30000.0,23900,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000162232,라운드랩 1025 독도 토너 300ml 듀오 기획,300,37000.0,25900,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000153627,에스트라 아토베리어365 로션 150ml 장벽짝꿍 기획,190,28000.0,22400,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...


In [113]:
today = datetime.today().strftime("%Y%m%d")
olive_df.to_csv("olive{today}.csv".format(today=today), index=False)
s3.upload_file('olive{today}.csv'.format(today=today), '2018102232','olive/{today}.csv'.format(today=today))